In [314]:
import PyPDF2
import tabula as tb
import pandas as pd
import re

In [315]:
file = 'Sectionnement 2022-Arrond 16 - AnnexeV3.pdf'

In [316]:
n = PyPDF2.PdfFileReader(open(file,mode='rb')).getNumPages()

In [317]:
#data = tb.read_pdf(file, area = (300, 0, 600, 800), pages = '2-3')
data = tb.read_pdf(file,  pages = f'2-{n}', lattice=True)

Got stderr: May 31, 2022 2:08:59 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
May 31, 2022 2:08:59 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
May 31, 2022 2:08:59 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider addTrueTypeCollection
java.io.IOException: Missing TTC header
	at org.apache.fontbox.ttf.TrueTypeCollection.<init>(TrueTypeCollection.java:73)
	at org.apache.fontbox.ttf.TrueTypeCollection.<init>(TrueTypeCollection.java:45)
	at org.apache.pdfbox.pdmodel.font.FileSystemFontProvider.addTrueTypeCollection(FileSystemFontProvider.java:618)
	at org.apache.pdfbox.pdmodel.font.FileSystemFontProvider.scanFonts(FileSystemFontProvider.java:383)
	at org.apache.pdfbox.pdmodel.font.FileSystemFontProvider.<init>(FileSystemFontProvider.java:358)
	at org.apache.pdfbox.pdmodel.font.FontMapperImpl$DefaultFontProvider.<clinit>(FontMapperImpl.java:140)
	at org.apache.pdfbox.pdmodel.font.FontMapperImpl.getProvider(FontMapperImpl.java:159)
	at or

In [319]:
len(data)

60

In [356]:
data2 = []
for i in data:
    i2=i.drop(0)
    data2.append(i2)
    
final = pd.concat(data2)

In [357]:
final = final.rename(columns={
                    final.columns[0]:"Quartier",
                    final.columns[1]:"Num",
                    final.columns[2]:"Désignation",
                    final.columns[3]:"Adresse",
                    final.columns[4]:"Zone"
}
            )
final=final.reset_index()
final=final.drop(["index"],axis=1)

In [358]:
final2 = final["Zone"].str.extract("([0-9]{1,3})( bis)?au([0-9]{1,3})( bis)?", expand=True)
final2 = final2.fillna("")
final["Zone1"],final["Zone2"]=(
    (final2[final2.columns[0]]+final2[final2.columns[1]]),
    (final2[final2.columns[2]]+final2[final2.columns[3]])
)

In [359]:
final["ZoneAdresse"]=final["Zone"].str.replace("([0-9]{1,3})( bis)?au([0-9]{1,3})( bis)?", "",regex=True)
final["ZoneAdresse"]=final["ZoneAdresse"].str.replace("\r", " ")
final["Désignation"]=final["Désignation"].str.replace("\r", " ")
final["Adresse"]=final["Adresse"].str.replace("\r", " ")
final1 = final.drop(["Zone"],axis=1)

In [360]:
final1

,Quartier,Num,Désignation,Adresse,Zone1,Zone2,ZoneAdresse
0,MUETTE,1,MAIRIE D'ARRONDISSEMENT,"71 AVENUE HENRI MARTIN, 75016 PARIS",71,97,AVENUE HENRI MARTIN
1,MUETTE,1,MAIRIE D'ARRONDISSEMENT,"71 AVENUE HENRI MARTIN, 75016 PARIS",1,5,AVENUE RODIN
2,MUETTE,1,MAIRIE D'ARRONDISSEMENT,"71 AVENUE HENRI MARTIN, 75016 PARIS",2,4,AVENUE RODIN
3,MUETTE,1,MAIRIE D'ARRONDISSEMENT,"71 AVENUE HENRI MARTIN, 75016 PARIS",48,60,BOULEVARD EMILE AUGIER
4,MUETTE,1,MAIRIE D'ARRONDISSEMENT,"71 AVENUE HENRI MARTIN, 75016 PARIS",63,71,RUE DE LA POMPE
...,...,...,...,...,...,...,...
1212,CHAILLOT,68,ECOLE ELEMENTAIRE,"21 RUE AMIRAL HAMELIN, 75016 PARIS",26,48,RUE PAUL VALERY
1213,CHAILLOT,68,ECOLE ELEMENTAIRE,"21 RUE AMIRAL HAMELIN, 75016 PARIS",1,7,RUE RUDE
1214,CHAILLOT,68,ECOLE ELEMENTAIRE,"21 RUE AMIRAL HAMELIN, 75016 PARIS",2,6,RUE RUDE
1215,CHAILLOT,68,ECOLE ELEMENTAIRE,"21 RUE AMIRAL HAMELIN, 75016 PARIS",1,11,VILLA D EYLAU


In [361]:
final1.to_csv("test2.csv")